In [1]:
from time import time
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

$$ 
\frac{\partial C}{\partial t} = D \frac{\partial^2 C}{\partial x^2} - u\frac{\partial C}{\partial x} + r
$$

In [542]:
def PFR(t, y, u, D, c_in, dx):
    """
    Differential equations for the PFR equations.

    The ODEs are derived using the method of lines.
    """
    # dydt is the return value of this function.
    dydt = np.zeros_like(y)

    # Compute dy/dt.  The end points and the interior points
    # are handled separately.
    dydt[:,0] = - u*(-c_in+y[:,1])/dx + D*(-2.0*y[:,0] + 2.0*y[:,1]) / dx**2
    dydt[:1:-1] = D*np.diff(y,2) / dx**2
    dydt[:,1:] -= u*np.diff(y)/dx
    dydt[:,-1]   = '''2.0*D*(- y[-1] + y[-2]) / dx**2''' - u*(-y[-1]+y[-2])/dx

    return dydt

In [546]:
x = np.linspace(0,1, num=50)
y0 = np.zeros_like(x)
y0[0]=1
# y0 = np.random.randn(5000)
t = np.linspace(0, 100, 11)
u = 0
D = 0.0001
c_in = 1
dx = 0.025

In [549]:
time0= time()
sol = solve_ivp(lambda t,y: PFR(t,y,u,D,c_in,dx), (0,100), y0, vectorized=True)# method='BDF')
print(time()-time0)

ValueError: could not broadcast input array from shape (50,0) into shape (48,1)

In [548]:
from ipywidgets import interact, widgets
@interact
def f(t=(0, len(sol.t)-1)):
    y = sol.y[:,t]
    plt.plot(y)
    print(f'{sol.t[t]:.2f}')

interactive(children=(IntSlider(value=13, description='t', max=26), Output()), _dom_classes=('widget-interact'…

In [537]:
sol.y[-1,:]

array([ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
       -6.63323188e-055, -1.34168082e-041, -2.28585473e-033,
       -1.88423727e-027, -1.14354246e-022, -1.11510081e-018,
       -2.60667243e-015, -1.96873801e-012, -5.89214108e-010,
       -1.35285274e-008, -2.04128053e-007, -2.24510937e-006,
       -1.94334180e-005, -1.39720189e-004, -8.90446607e-004,
       -5.38064336e-003, -3.27368944e-002, -2.15623602e-001,
       -1.59983659e+000, -1.27280921e+001, -7.99955751e+001,
       -5.07121703e+002, -3.21274828e+003, -2.45814852e+004,
       -2.40797979e+005, -2.71788563e+006, -1.80922889e+007,
       -1.31321518e+008, -1.12478960e+009, -1.06237354e+010,
       -1.00462950e+011, -9.50024075e+011, -8.98386659e+012,
       -8.49555932e+013, -8.03379341e+014, -7.75505339e+015,
       -7.48598451e+016, -7.22625123e+017, -6.97552964e+018,
       -6.73350707e+019,